# Accessing an AutoAI Model
In this notebook, we use the Watson Machine Learning (WML) API to find the available models available, and find the availablr deployments.

We then score some records using a Churn model.

Finally, we show how a deployment could be removed.

See:<br/>https://wml-api-pyclient-dev-v4.mybluemix.net/<br/>
https://watson-ml-v4-api.mybluemix.net/<br/>
https://wml-api-pyclient.mybluemix.net/

In [ ]:
# Install the Watson Machine Learning API client
!pip install watson-machine-learning-client-v4 > wmlclientpip.out

In [ ]:
wml_credentials = {
  "apikey": "",
  "instance_id": "",
  "url": ""
}

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# List models already in the repository
client.repository.list_models()

## Accessing the deployed model
We extract the information on deployed models and find the deployment for **`AutoAI Churn Deployed`**.

In [ ]:
# List the deployed models
client.deployments.list()

In [ ]:
# Extract the information for "Churn model deployment" 
deployments_details = client.deployments.get_details()
deployed_uid = next(item for item in deployments_details['resources'] 
                    if item['entity']["name"] == "AutoAI Churn Deployed")['metadata']['guid']

### Score a record

In [ ]:
# Execute the model
scoring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA: 
                   [{
                     'fields': ['ID','Gender','Status','Children','Est Income','Car Owner',
                                'Age','LongDistance','International','Local','Dropped',
                                'Paymethod','LocalBilltype','LongDistanceBilltype',
                                'Usage','RatePlan'], 
                     'values': [[1,'F','S',1.0,38000.0,'N',24.393333,23.56,0.0,206.08,0.0,'CC','Budget','Intnl_discount',229.64,3.0],                      
                                [6,'M','M',2.0,29616.0,'N',49.426667,29.78,0.0,45.5,0.0,  'CH','FreeLocal','Standard',75.29,2.0]
                               ]
                    }]
                  }
predictions = client.deployments.score(deployed_uid, scoring_payload)


In [ ]:
predictions

In [ ]:
for prediction in predictions['predictions'] :
    for result in prediction['values'] :
        print('Prediction: ' + str(result[0]) + ", probability: [" + 
              str(result[1][0]) + ', ' +  str(result[1][1]) + "]" )


## Removing a deployed model
We can remove a model from the repository using the remove method.
In the example below, we remove the deployed model we just used since we already have the uid.
We could look over like we did before and remove all the deployed models.

A similar delete operation can be used to remove the saved models.

In [ ]:
# Display the list first
client.deployments.list()

In [ ]:
# Remove the deployment
client.deployments.delete(deployed_uid)

In [ ]:
client.deployments.list()